#  GPT-2 Generation and Fine-Tuning

This notebook explores GPT-2 (Generative Pretrained Transformer-2) from OpenAI. Read more about it [here](https://openai.com/blog/better-language-models/).

Activities include:

0. Setup
1. Generate samples from pre-trained gpt-2 model
2. Fine-tune gpt-2 on text of your choosing. 

Adapted by Robert Twomey (rtwomey@unl.edu) for Machine Learning for the Arts SP22 from this [google colab](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) by [Max Woolf](http://minimaxir.com). See his repo [gpt-2-simple](https://github.com/minimaxir/gpt-2-simple).

# 0. Setup

Run once to install the library

In [5]:
%pip install -q gpt-2-simple

Note: you may need to restart the kernel to use updated packages.


restart the kernel and run the imports

RUN THIS EACH TIME

# 0.5 Imports: run every time

In [6]:
import gpt_2_simple as gpt2
import tensorflow as tf
from datetime import datetime

## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

DONT NEED TO RUN THIS EACH TIME

In [7]:
!nvidia-smi

Tue Feb 15 11:01:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:D8:00.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

note the memory usage (0MiB / 32510MiB) for the Tesla V100.
You can re-rerun the above cell to see what memory your code/models are using during training.

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are four released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the the current working directory at `/models/<model_name>`.

RUN THIS

In [8]:
model_name = "355M" # largest model we can fine-tune

run once to download the file

don't need to do this again:

In [16]:
gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 959Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 4.76Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 1.68Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:23, 61.2Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 1.15Git/s]                                               
Fetching model.ckpt.meta: 1.05Mit [00:00, 4.22Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 6.20Mit/s]                                                      


# 1. Generate Text From The Pretrained Model

If you want to generate text from the pretrained model pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`. (This is currently the only way to generate text from the 774M or 1558M models with this notebook.)

In [9]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

2022-02-15 11:01:31.613578: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-15 11:01:32.255160: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


Loading pretrained model models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


## Sample from the model
The follow cell samples from gpt-2, using the provided prefix (seed) and other parameters. It starts the TF session and generates the samples.

Try changing the parameters below to change the output: 
- `prefix` is the prompt. This will be the starting string/seed for your generation. Use your own text. 
- `temperature` sets the variability/randomness of the output. Range 0.0-1.0
- `length` sets the lenght of output (in tokens). max is 1024.


In [10]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="My cat's name is " ,
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

My cat's name is _________ and she's been a member of the ___________ for about three years.

I was looking through your website and found a link to your blog. I'm sure you know what I'm talking about. You have a cat. She's been a member of the ___________ for about three years. She's very sweet and her name is _________. She's a great cat. I love her dearly and I love to pet her. She's very loyal. She has
My cat's name is ____. She's a girl. She's about 10 years old. She's very sweet and sweet-natured. She's always happy, and she's very funny. She likes to play with me. I always say, "She's a very sweet girl."

She's a big girl, but she's also a little girl. She's about three or four. She's a little boy. She's about two or three years old. She's a little girl. She's
My cat's name is ____ and I'm an ____."

The following text messages were sent to the website:

"It's so good to meet you again. I'm just wondering how you feel about the way I look at you. You're a nice guy, so I can't de

## Activities
- try varying the prefix. 
  - what length of prefix works best with the given model? 
  - how does the choice of prefix change the format/form of the output.
- try varying the temperature.
- try loading the different sized models (124M, 355M, 774M, 1558M) and generate text without changing the other parameters. 
  - Do you notice any qualitative differences in the output? 

# 2. Fine-Tuning GPT-2

If you have already generated with gpt2, you need to reset the tf graph and gpt2 session. Otherwise, we create a new one:

In [11]:
#model_name = "355M" # same model as selected above

tf.compat.v1.reset_default_graph()

# check if sess exists (e.g. if we ran section 1 above)
var_exists = 'sess' in locals() or 'sess' in globals()

if not var_exists:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

2022-02-15 11:01:51.885437: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


## Upload a text file
For this, we will use a text file you provide to finetune (continue training) GPT-2. You can use any plain text (.txt) file. 

Simply drag and dropy our text file into the file browser at left. 

Once you have uploaded your file, update the file name in the cell below, then run it.

In [12]:
file_name = "cats.txt" # your file here

## Run the finetuning

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every `save_every` steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them. If your input text is smaller, training might proceed more quickly.

Other optional-but-helpful parameters for `gpt2.finetune`:

*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [13]:
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=150,            #depending on size of text this changes (maybe 1000?)
              restore_from='fresh', # change to 'latest' to resume
              run_name='run1',
              print_every=10,
              learning_rate=1e-5,
              #learning rate smaller, learns less per step
              sample_every=50,
              save_every=100
              )

For larger models, the recommended finetune() parameters are:
	use_memory_saving_gradients = True
	only_train_transformer_layers = True
	accumulate_gradients = 1



2022-02-15 11:02:03.932660: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00, 2676.65it/s]

dataset has 9182 tokens
Training...


[10 | 25.75] loss=2.44 avg=2.44
[20 | 45.77] loss=2.42 avg=2.43
[30 | 65.80] loss=2.15 avg=2.33
[40 | 85.85] loss=2.00 avg=2.25
[50 | 105.91] loss=1.69 avg=2.14
======== SAMPLE 1 ========
 got a load of this girl on her first time, and it was an absolute blast, she's such a sweetheart! She's petite, and her body is just incredible too…I can't wait to see how big she gets!

This girl is actually a gorgeous redhead with a sexy face, good weight, and a mature figure! She's a sweet girl who just wants love and attention, she really needs everyone to love and respect her and take pride in her, she deserves it!

We are big fans of this beautiful girl, she had some super short hairs growing out of her cutie mark a couple of months back but she was so petite her lady-parts were too long for her big boobies, they looked so cute and squishy and round and round! These beautiful cuties were removed a short time ago, they can look so cute but when they are cut they become SO much more…disgusting! W

## Notes on finetuning

Keep an eye on the loss, and how quickly it is dropping. A too-rapid drop in loss could be a sign of overfitting, and a learning rate (lr) that is too high. 

After the model is trained, you can download the checkpoint folder to save your work. Training checkpoints are saved to `checkpoint/run1` (or whatever you chose for the run name above).

You can compress it to a rar file and download that. Ask the instructor how.

You're done! Feel free to go to the Generate Text From The Trained Model section to generate text based on your retrained model.

## Finetune some more

If you have already generated with gpt2, you need to reset the tf graph and gpt2 session. Otherwise, we create a new one:

In [12]:
#model_name = "355M" # same model as selected above

tf.compat.v1.reset_default_graph()

# check if sess exists (e.g. if we ran section 1 above)
var_exists = 'sess' in locals() or 'sess' in globals()

if not var_exists:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

2022-02-10 11:18:35.772450: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:5e:00.0, compute capability: 7.0


To fine-tune some more, run the following. Be sure to increase the number of steps (if it was `500` before, change to `1000` to train for 500 more. the number is cumulative).

In [13]:
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=20,
              restore_from='latest', # change to 'latest' to resume
              run_name='run1',
              print_every=10,
              learning_rate=1e-5,
              sample_every=100,
              save_every=100
              )

For larger models, the recommended finetune() parameters are:
	use_memory_saving_gradients = True
	only_train_transformer_layers = True
	accumulate_gradients = 1

Loading checkpoint checkpoint/run1/model-200
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-200
Loading dataset...


100%|██████████| 1/1 [00:00<00:00, 4369.07it/s]

dataset has 4906 tokens
Training...
Saving checkpoint/run1/model-200


======== SAMPLE 1 ========
 a apron; a bed; bedspread; bunk; bunk berth; bunkhouse; bunkhouse; bedroll; blanket; blanket of leaves; blanket; blanket of sea; blanket of ashes; blanket of sun; Christmas; Christmas Day; Christmas Eve; Christmas Eve Party; China; Christmas goose; Christmas greeting card; Christmas sweater; Christmas, New Zealand; Christmas tree; costume; dinner gown; dinner dress; dinner suit; dishdasha; dishroast; dishwasher; dishwashing machine; dung beetle; dance; Earth; English bulldog; English bulldog mix; English bulldog mix bred for; English bulldog mix/breed; English bulldog/dutch mix; English bulldog/German mix; English bulldog/American Bulldog mix; fire ant; football; First World War; French bulldog; German shepherd; hamster; hiker (biking in the dark); hodgkin's lymphoma; hound; Inuit; Japanese knothead steelhead; lunch; malarial fever; Maypole Day; Mount Everest; pig's lung; plover; poochie; ranch house; samurai; squirrel; snow; spoon; spanish bulldog mix; stin

KeyboardInterrupt: 

# 3. Generate Text From The Finetuned Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [14]:
gpt2.generate(sess, run_name='run1') # no prefix, unconditional generation, tells you what it has learned without prefix

Nigel Farage has criticised the Government over its decision to cut off Leicestershire Council's funding.

Mr Farage told the BBC's Sunday Politics the cuts will have a "huge effect" on the community.

He said: "We're in a very difficult position.

"We've had a real good relationship with the council but now from the beginning of next year the amount we get will be cut by 40%.

"We don't know what the consequences are and it's very difficult for the local people who live in the area.

"We will almost certainly lose our council tax-free status by the end of next year."

We don't know what the consequences are and it's very difficult for the local people who live in the area Nigel Farage
Leicestershire Council
The council said it had received this funding immediately after the election.

Chief executive Mark Jones said: "We are very concerned by the Government's decision to cut funding to Leicestershire Council immediately after the election.

"We have been in close touch with Leicesters

In [15]:
gpt2.generate(sess, run_name='run1', prefix="Baby")

Baby baby. You're so cute when you're happy.

Your smile is the best thing to ever happen to my soul.

You are my best friend.

You will always be in my heart.

I will love and care for you like I've never loved you before.

You will always be in my prayers.

You will always be in my thoughts.
I will always be in my prayers.

You will forever be in my thoughts.

I will forever be in your prayers.

I will forever be in your prayers.

You will forever be in my thoughts.

I will forever be in your prayers.

You will forever be in my thoughts.

I will forever be in your prayers.

You will forever be in my thoughts.

You will forever be in my thoughts.

You will forever be in my thoughts.

You will forever be in my thoughts.

You will forever be in my thoughts.

You will forever be in my thoughts.

You will forever be in my thoughts.

You will forever be in my thoughts.

You will forever be in my thoughts.

You will forever be in my thoughts.

You will forever be in my thoughts.

You will f

## Notes
If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [18]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="Toby",
              nsamples=5,
              batch_size=5
              )

TobyDaniels

SpongeBob SquarePants
Tom Khaos
This guy is an absolute beast. He is almost 6 years old and weighs about 100 pounds. He is very affectionate and super playful. He likes to sit on your lap and snuggle against you, or you can lean into him and watch him go berserk in pure bliss. He is super playful and energetic and will have an instant fan-base. He will be a huge hit on the bridge!

Kiki
Scout
SpongeBob SquarePants
Kiki is a golden retriever mix that was born in Orange County, CA. She was the favorite of the kennels when she was a young kitten, and still is today. She is a sweet, affectionate gal, with a lot of energy. She loves to chase her toys, and to sit down and enjoy long, quiet naps with humans. She is well-mannered and tidy, and would do well in many homes.

Salty
English Bulldog
SpongeBob SquarePants
This is a gorgeous little guy! Salty is a very affectionate bulldog mix. He is very easy-going around people, and
Toby, the cat, was kidnapped recently along with his 

In [39]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

Download the file by hand in the browser at left.

## Load a Trained Model Checkpoint

Uploaded your saved checkpoint and unzip it.

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

This will reset or start the tensorflow session as necessary.

In [17]:
tf.compat.v1.reset_default_graph()

if not sess:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

gpt2.load_gpt2(sess, run_name='run1')

2022-02-10 11:21:42.332368: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:5e:00.0, compute capability: 7.0


Loading checkpoint checkpoint/run1/model-200
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-200


# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# License

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Reference
- Max's [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!
- Original repo: [gpt-2-simple](https://github.com/minimaxir/gpt-2-simple) by [Max Woolf](http://minimaxir.com). 
- Original [google colab](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) from Max.